<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Advanced/CollectData.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install aitk --upgrade --quiet

In [1]:
import aitk.robots
aitk.robots.__version__

'0.8.0'

### Collect wall following data

Create a world with several obstacles.  Use a controller to follow walls on the robot's left
side.  Collect the robot's IR data and movements so as to eventually be able to train a neural
network to learn how to wall follow. Save the data to a CSV file.

In [2]:
world = aitk.robots.World(width=200, height=200)
world.add_wall("blue", 0, 0, 50, 50)
world.add_wall("blue", 75, 200, 125, 150)
robot = aitk.robots.Scribbler()
world.add_robot(robot)
robot.add_device(aitk.robots.RangeSensor(width=45,max=20,name="front"))
robot.add_device(aitk.robots.RangeSensor(width=45,max=20,position=(6,-6),
                                         a=90,name="front-left"))
robot.add_device(aitk.robots.RangeSensor(width=45,max=20,position=(-6,-6),
                                         a=90,name="back-left"))
robot.set_pose(100,100,0)

Random seed set to: 1573328


In [3]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [4]:
def scale(v, max):
    return v/max

def wall_follow(robot):
    """Find and follow a wall on the robot's left side"""
    f = robot["front"].get_distance()
    fl = robot["front-left"].get_distance()
    bl = robot["back-left"].get_distance()
    max_dist = robot["front"].get_max()
    readings = [f, fl, bl]
    inputs = [scale(v, max_dist) for v in readings]
    tolerance = 1.5
    difference = fl - bl
    translate = None
    rotate = None
    situation = None
    if f < max_dist:
        # wall in front, turn right
        translate = 0.05
        rotate = -0.3
        situation = "blocked"
        robot.speak("blocked")
    elif fl < max_dist or bl < max_dist:
        # left side is against a wall
        if abs(difference) > tolerance and difference < 0:
            # keep left side aligned with wall
            translate = 0.3
            rotate = -1
            situation = "adjustR"
            robot.speak("adjustR")
        elif abs(difference) > tolerance and difference > 0:
            # keep left side aligned with wall
            translate = 0.3
            rotate = 1
            situation = "adjustL"
            robot.speak("adjustL")
        elif min(fl, bl) < 10:
            # robot is too close to wall, move right
            translate = 0.5
            rotate = -0.1
            situation = "close"
            robot.speak("close")
        elif max(fl, bl) > 15:
            # robot is too far from wall, move left
            translate = 0.2
            rotate =0.4
            situation = "far"
            robot.speak("far")
        else:
            # alignment is good, go straight
            translate = 0.5
            rotate = 0
            robot.move(0.5, 0)
            situation = "follow"
            robot.speak("follow")
    else:
        # no wall sensed, move forward to find a wall
        translate = 1.0
        rotate = 0.0
        situation = "no_wall"
        robot.speak("no_wall")
    if abs(rotate) > 1:
        print(rotate)
    robot.move(translate, rotate)
    targets = [translate, rotate, situation]
    data.append(inputs + targets)

### Try several random starting points

Collect data from many different random poses so as to create a good variety of data.

In [5]:
from random import randrange
data = []

for i in range(10):
    robot.set_pose(randrange(50, 150), 
                   randrange(90,110), 
                   randrange(0,360))
    world.seconds(25, [wall_follow], real_time=False, quiet=True)

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(data, columns=['F_ir', 'FL_ir', 'BL_ir', 'translate', 'rotate', 'situation'])

In [8]:
df['situation'].value_counts()

follow     658
adjustR    437
blocked    385
no_wall    349
far        344
adjustL    320
close        7
Name: situation, dtype: int64

In [9]:
df.to_csv("follow_data.csv")